In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.appName("MyApp").getOrCreate()

23/11/07 21:51:36 WARN Utils: Your hostname, DESKTOP-TCH95BD resolves to a loopback address: 127.0.1.1; using 192.168.233.120 instead (on interface eth0)
23/11/07 21:51:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/07 21:51:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Infos fato_Filmes

In [4]:
fatos_filmes = spark.read.parquet("file:/home/hadoop/imdb/refined/fato_filmes/*.parquet")
fatos_filmes.printSchema()
fatos_filmes.show(10)
contagem_de_dados = fatos_filmes.count()
contagem_de_colunas = len(fatos_filmes.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

root
 |-- id_filme: string (nullable = true)
 |-- id_nome: string (nullable = true)
 |-- titulo: string (nullable = true)
 |-- lancamento: string (nullable = true)
 |-- duracao_filme: string (nullable = true)
 |-- genero: string (nullable = true)

+---------+---------+--------------------+----------+-------------+--------------------+
| id_filme|  id_nome|              titulo|lancamento|duracao_filme|              genero|
+---------+---------+--------------------+----------+-------------+--------------------+
|tt0063803|nm0000217|Who's That Knocki...|      1967|           90|               Drama|
|tt0063803|nm1058895|Who's That Knocki...|      1967|           90|               Drama|
|tt0068309|nm0718639|       Boxcar Bertha|      1972|           88| Crime,Drama,Romance|
|tt0068309|nm0181020|       Boxcar Bertha|      1972|           88| Crime,Drama,Romance|
|tt0068309|nm0181019|       Boxcar Bertha|      1972|           88| Crime,Drama,Romance|
|tt0070379|nm0000217|        Mean Street

In [3]:
basicos_filmes = spark.read.parquet("file:/home/hadoop/imdb/parquet/basico_filmes_drama/2023/11/01/*.parquet")
basicos_filmes.printSchema()
basicos_filmes.show(10)
contagem_de_dados = basicos_filmes.count()
contagem_de_colunas = len(basicos_filmes.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

root
 |-- id_filme: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- titulo: string (nullable = true)
 |-- titulo_original: string (nullable = true)
 |-- adulto: string (nullable = true)
 |-- lancamento: string (nullable = true)
 |-- fim_lancamento: string (nullable = true)
 |-- duracao_filme: string (nullable = true)
 |-- genero: string (nullable = true)

+---------+-----+--------------------+--------------------+------+----------+--------------+-------------+---------------+
| id_filme| tipo|              titulo|     titulo_original|adulto|lancamento|fim_lancamento|duracao_filme|         genero|
+---------+-----+--------------------+--------------------+------+----------+--------------+-------------+---------------+
|tt0000591|movie|    The Prodigal Son|   L'enfant prodigue|     0|      1907|            \N|           90|          Drama|
|tt0000615|movie|  Robbery Under Arms|  Robbery Under Arms|     0|      1907|            \N|           \N|          Drama|
|tt000063

In [ ]:
# Infos Diretores e Escritores

In [3]:
diretores_escritores = spark.read.parquet("file:/home/hadoop/imdb/parquet/diretores_escritores/2023/11/01/*.parquet")

contagem_de_dados = diretores_escritores.count()
contagem_de_colunas = len(diretores_escritores.columns)

diretores_escritores.printSchema()
diretores_escritores.show(10)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

root
 |-- id_filme: string (nullable = true)
 |-- diretores: string (nullable = true)
 |-- escritores: string (nullable = true)



+---------+-------------------+----------+
| id_filme|          diretores|escritores|
+---------+-------------------+----------+
|tt0000001|          nm0005690|        \N|
|tt0000002|          nm0721526|        \N|
|tt0000003|          nm0721526|        \N|
|tt0000004|          nm0721526|        \N|
|tt0000005|          nm0005690|        \N|
|tt0000006|          nm0005690|        \N|
|tt0000007|nm0005690,nm0374658|        \N|
|tt0000008|          nm0005690|        \N|
|tt0000009|          nm0085156| nm0085156|
|tt0000010|          nm0525910|        \N|
+---------+-------------------+----------+
only showing top 10 rows

O DataFrame tem 3 colunas.
O DataFrame tem 10265844 linhas (dados).


In [ ]:
# JOIN entre info_filmes e info_diretores_escritores

In [4]:
basicos_filmes.createOrReplaceTempView("info_filmes")
diretores_escritores.createOrReplaceTempView("info_diretores_escritores")


filmes_martin_sql = """
SELECT
    f.id_filme,
    f.titulo,
    f.lancamento,
    f.duracao_filme,
    f.genero,
    de.escritores
FROM
    info_filmes f
JOIN
    info_diretores_escritores de
ON
    f.id_filme = de.id_filme
WHERE
    de.diretores = 'nm0000217'
"""

filmes_diretores_escritores = spark.sql(filmes_martin_sql)

# removendo o id do filme Roosevelt pois ainda está em produção

id_filme_a_remover = 'tt7428530'

filmes_diretores_escritores = filmes_diretores_escritores.filter(filmes_diretores_escritores["id_filme"] != id_filme_a_remover)

filmes_diretores_escritores.show(30)

# contagem de colunas

contagem_de_dados = filmes_diretores_escritores.count()
contagem_de_colunas = len(filmes_diretores_escritores.columns)

print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

+---------+--------------------+----------+-------------+--------------------+--------------------+
| id_filme|              titulo|lancamento|duracao_filme|              genero|          escritores|
+---------+--------------------+----------+-------------+--------------------+--------------------+
|tt0063803|Who's That Knocki...|      1967|           90|               Drama| nm0000217,nm1058895|
|tt0068309|       Boxcar Bertha|      1972|           88| Crime,Drama,Romance|nm0718639,nm01810...|
|tt0070379|        Mean Streets|      1973|          112|Crime,Drama,Thriller| nm0000217,nm0552731|
|tt0071115|Alice Doesn't Liv...|      1974|          112|       Drama,Romance|           nm0315205|
|tt0075314|         Taxi Driver|      1976|          114|         Crime,Drama|           nm0001707|
|tt0076451|  New York, New York|      1977|          155| Drama,Music,Musical| nm0711996,nm0552731|
|tt0081398|         Raging Bull|      1980|          129|Biography,Drama,S...|nm0483766,nm01417...|


O DataFrame tem 6 colunas.
O DataFrame tem 23 linhas (dados).


In [ ]:
# Transformando a coluna escritores, usando split e explode para gerar atomicidade

In [7]:
from pyspark.sql.functions import split, col, explode

# dividir a coluna 'id_filme' em uma lista de valores

filmes_diretores_escritores = filmes_diretores_escritores.withColumnRenamed("escritores", "id_nome")

filmes_diretores_escritores = filmes_diretores_escritores.withColumn("id_nome", split(col("id_nome"), ","))

# explode para criar uma linha para cada valor na lista 'id_filme'

filmes_diretores_escritores_exploded = filmes_diretores_escritores.withColumn("id_nome", explode(col("id_nome")))

# contagem de colunas

contagem_de_dados = filmes_diretores_escritores_exploded.count()
contagem_de_colunas = len(filmes_diretores_escritores_exploded.columns)

filmes_diretores_escritores_exploded.show(10)

print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")


+---------+--------------------+----------+-------------+--------------------+---------+
| id_filme|              titulo|lancamento|duracao_filme|              genero|  id_nome|
+---------+--------------------+----------+-------------+--------------------+---------+
|tt0063803|Who's That Knocki...|      1967|           90|               Drama|nm0000217|
|tt0063803|Who's That Knocki...|      1967|           90|               Drama|nm1058895|
|tt0068309|       Boxcar Bertha|      1972|           88| Crime,Drama,Romance|nm0718639|
|tt0068309|       Boxcar Bertha|      1972|           88| Crime,Drama,Romance|nm0181020|
|tt0068309|       Boxcar Bertha|      1972|           88| Crime,Drama,Romance|nm0181019|
|tt0070379|        Mean Streets|      1973|          112|Crime,Drama,Thriller|nm0000217|
|tt0070379|        Mean Streets|      1973|          112|Crime,Drama,Thriller|nm0552731|
|tt0071115|Alice Doesn't Liv...|      1974|          112|       Drama,Romance|nm0315205|
|tt0075314|         T

In [ ]:
# CRIANDO UMA TABELA FATO_FILMES COM ESSAS TRANSFORMAÇÕES

In [8]:
filmes_diretores_escritores_exploded.createOrReplaceTempView("fato_filmes")

fato_filmes_sql = """
SELECT
    id_filme,
    id_nome,
    titulo,
    lancamento,
    duracao_filme,
    genero

FROM
    fato_filmes
"""

fato_filmes = spark.sql(fato_filmes_sql)

contagem_de_dados = fato_filmes.count()
contagem_de_colunas = len(fato_filmes.columns)

print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

fato_filmes.show(50)

O DataFrame tem 6 colunas.
O DataFrame tem 49 linhas (dados).


+---------+---------+--------------------+----------+-------------+--------------------+
| id_filme|  id_nome|              titulo|lancamento|duracao_filme|              genero|
+---------+---------+--------------------+----------+-------------+--------------------+
|tt0063803|nm0000217|Who's That Knocki...|      1967|           90|               Drama|
|tt0063803|nm1058895|Who's That Knocki...|      1967|           90|               Drama|
|tt0068309|nm0718639|       Boxcar Bertha|      1972|           88| Crime,Drama,Romance|
|tt0068309|nm0181020|       Boxcar Bertha|      1972|           88| Crime,Drama,Romance|
|tt0068309|nm0181019|       Boxcar Bertha|      1972|           88| Crime,Drama,Romance|
|tt0070379|nm0000217|        Mean Streets|      1973|          112|Crime,Drama,Thriller|
|tt0070379|nm0552731|        Mean Streets|      1973|          112|Crime,Drama,Thriller|
|tt0071115|nm0315205|Alice Doesn't Liv...|      1974|          112|       Drama,Romance|
|tt0075314|nm0001707|

In [ ]:
# Exportando essa tabela para o Dashboard

In [29]:
from datetime import datetime

current_date = datetime.now().strftime("%Y/%m/%d")

# Definindo a variável do Output

output_path = f"file:/home/hadoop/imdb/refined/fato_filmes"

# Salvar o DataFrame como um arquivo Parquet

fato_filmes.coalesce(1).write.saveAsTable(name="fato_filmes",mode="overwrite",path=output_path,format="parquet")

print("Os dados foram salvos com sucesso como arquivo Parquet.")

Os dados foram salvos com sucesso como arquivo Parquet.


In [ ]:
# INFO colaboradores - dim_equipe_filmes

In [25]:
equipe = spark.read.parquet("file:/home/hadoop/imdb/parquet/info_colaboradores/2023/11/02/*.parquet")

contagem_de_dados = equipe.count()
contagem_de_colunas = len(equipe.columns)
# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

equipe.show(10)

O DataFrame tem 6 colunas.
O DataFrame tem 12954255 linhas (dados).
+---------+---------------+--------------------+--------------------+--------------+---------------+
|  id_nome|           nome| participacao_filmes| profissao_principal|ano_nascimento|ano_falecimento|
+---------+---------------+--------------------+--------------------+--------------+---------------+
|nm0000001|   Fred Astaire|tt0072308,tt00319...|soundtrack,actor,...|          1899|           1987|
|nm0000002|  Lauren Bacall|tt0038355,tt01170...|  actress,soundtrack|          1924|           2014|
|nm0000003|Brigitte Bardot|tt0054452,tt00564...|actress,soundtrac...|          1934|        vivo(a)|
|nm0000004|   John Belushi|tt0072562,tt00804...|actor,soundtrack,...|          1949|           1982|
|nm0000005| Ingmar Bergman|tt0050986,tt00839...|writer,director,a...|          1918|           2007|
|nm0000006| Ingrid Bergman|tt0034583,tt00368...|actress,soundtrac...|          1915|           1982|
|nm0000007|Humphrey Bog

In [ ]:
# algumas transformações

In [26]:
from pyspark.sql.functions import split, col, explode

# dividir a coluna 'id_filme' em uma lista de valores

equipe = equipe.withColumnRenamed("participacao_filmes", "id_filme")

contagem_de_dados = equipe.count()
contagem_de_colunas = len(equipe.columns)
# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

equipe.show(10)

O DataFrame tem 6 colunas.
O DataFrame tem 12954255 linhas (dados).
+---------+---------------+--------------------+--------------------+--------------+---------------+
|  id_nome|           nome|            id_filme| profissao_principal|ano_nascimento|ano_falecimento|
+---------+---------------+--------------------+--------------------+--------------+---------------+
|nm0000001|   Fred Astaire|tt0072308,tt00319...|soundtrack,actor,...|          1899|           1987|
|nm0000002|  Lauren Bacall|tt0038355,tt01170...|  actress,soundtrack|          1924|           2014|
|nm0000003|Brigitte Bardot|tt0054452,tt00564...|actress,soundtrac...|          1934|        vivo(a)|
|nm0000004|   John Belushi|tt0072562,tt00804...|actor,soundtrack,...|          1949|           1982|
|nm0000005| Ingmar Bergman|tt0050986,tt00839...|writer,director,a...|          1918|           2007|
|nm0000006| Ingrid Bergman|tt0034583,tt00368...|actress,soundtrac...|          1915|           1982|
|nm0000007|Humphrey Bog

In [ ]:
# USANDO EXPLODE PARA ATOMIZAR

In [27]:
from pyspark.sql.functions import split, col, explode

# dividir a coluna 'id_filme' em uma lista de valores

equipe = equipe.withColumn("id_filme", split(col("id_filme"), ","))

# explode para criar uma linha para cada valor na lista 'id_filme'

equipe_ok = equipe.withColumn("id_filme", explode(col("id_filme")))

contagem_de_dados = equipe_ok.count()
contagem_de_colunas = len(equipe_ok.columns)
# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

equipe_ok.show(10)

O DataFrame tem 6 colunas.
O DataFrame tem 22699736 linhas (dados).
+---------+---------------+---------+--------------------+--------------+---------------+
|  id_nome|           nome| id_filme| profissao_principal|ano_nascimento|ano_falecimento|
+---------+---------------+---------+--------------------+--------------+---------------+
|nm0000001|   Fred Astaire|tt0072308|soundtrack,actor,...|          1899|           1987|
|nm0000001|   Fred Astaire|tt0031983|soundtrack,actor,...|          1899|           1987|
|nm0000001|   Fred Astaire|tt0050419|soundtrack,actor,...|          1899|           1987|
|nm0000001|   Fred Astaire|tt0053137|soundtrack,actor,...|          1899|           1987|
|nm0000002|  Lauren Bacall|tt0038355|  actress,soundtrack|          1924|           2014|
|nm0000002|  Lauren Bacall|tt0117057|  actress,soundtrack|          1924|           2014|
|nm0000002|  Lauren Bacall|tt0075213|  actress,soundtrack|          1924|           2014|
|nm0000002|  Lauren Bacall|tt003

In [ ]:
# Join da equipe que trabalha nos filmes do martin

In [28]:
equipe_ok.createOrReplaceTempView("dim_equipe_filmes")
fatos_filmes.createOrReplaceTempView("fato_filmes")

dim_equipe_sql = """
SELECT
    f.id_filme,
    f.id_nome,
    f.titulo,
    de.nome,
    de.profissao_principal,
    de.ano_nascimento,
    de.ano_falecimento

FROM
    fato_filmes f
JOIN
    dim_equipe_filmes de
ON
    f.id_filme = de.id_filme
ORDER BY
    titulo
"""

dim_equipe_filmes = spark.sql(dim_equipe_sql)

dim_equipe_filmes.show()

contagem_de_dados = dim_equipe_filmes.count()
contagem_de_colunas = len(dim_equipe_filmes.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")



+---------+---------+-----------+--------------------+--------------------+--------------+---------------+
| id_filme|  id_nome|     titulo|                nome| profissao_principal|ano_nascimento|ano_falecimento|
+---------+---------+-----------+--------------------+--------------------+--------------+---------------+
|tt0088680|nm0591387|After Hours|     Cuadro Flamenco|          soundtrack|            \N|        vivo(a)|
|tt0088680|nm0591387|After Hours|  Elizabeth Benjamin|       miscellaneous|            \N|        vivo(a)|
|tt0088680|nm0591387|After Hours|         Tommy Boyce|soundtrack,music_...|          1939|           1994|
|tt0088680|nm0591387|After Hours|Rosie & The Origi...|          soundtrack|            \N|        vivo(a)|
|tt0088680|nm0591387|After Hours|         Verna Bloom|             actress|          1938|           2019|
|tt0088680|nm0591387|After Hours|The Academy of An...|soundtrack,music_...|            \N|        vivo(a)|
|tt0088680|nm0591387|After Hours|    

O DataFrame tem 7 colunas.
O DataFrame tem 10080 linhas (dados).


In [ ]:
# Exportando essa tabela

In [29]:
from datetime import datetime

current_date = datetime.now().strftime("%Y/%m/%d")

# Definindo a variável do Output

output_path = f"file:/home/hadoop/imdb/refined/dim_equipe_filmes"

# Salvar o DataFrame como um arquivo Parquet

dim_equipe_filmes.coalesce(1).write.saveAsTable(name="dim_equipe_filmes",mode="overwrite",path=output_path,format="parquet")

print("Os dados foram salvos com sucesso como arquivo Parquet.")

Os dados foram salvos com sucesso como arquivo Parquet.


In [ ]:
# INFOS Filmes_atores

In [18]:
filme_ator = spark.read.parquet("file:/home/hadoop/imdb/parquet/filme_ator/2023/11/01/*.parquet")
filme_ator.printSchema()
filme_ator.show(10)
contagem_de_dados = filme_ator.count()
contagem_de_colunas = len(filme_ator.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

root
 |-- id_filme: string (nullable = true)
 |-- id_nome: string (nullable = true)
 |-- ordem: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- personagem: string (nullable = true)
 |-- trabalho: string (nullable = true)



+---------+---------+-----+---------------+----------+--------------------+
| id_filme|  id_nome|ordem|      categoria|personagem|            trabalho|
+---------+---------+-----+---------------+----------+--------------------+
|tt0000001|nm1588970|    1|           self|  ["Self"]|                  \N|
|tt0000001|nm0005690|    2|       director|        \N|                  \N|
|tt0000001|nm0374658|    3|cinematographer|        \N|director of photo...|
|tt0000002|nm0721526|    1|       director|        \N|                  \N|
|tt0000002|nm1335271|    2|       composer|        \N|                  \N|
|tt0000003|nm0721526|    1|       director|        \N|                  \N|
|tt0000003|nm1770680|    2|       producer|        \N|            producer|
|tt0000003|nm1335271|    3|       composer|        \N|                  \N|
|tt0000003|nm5442200|    4|         editor|        \N|                  \N|
|tt0000004|nm0721526|    1|       director|        \N|                  \N|
+---------+-

In [ ]:
# Join com Fato Filmes

In [19]:
filme_ator.createOrReplaceTempView("dim_atores")

dim_func_sql = """
SELECT
    f.id_filme,
    f.id_nome,
    f.titulo,
    de.ordem,
    de.categoria,
    de.personagem,
    de.trabalho

FROM
    fato_filmes f
JOIN
    dim_atores de
ON
    f.id_filme = de.id_filme
"""

# Agora você pode executar consultas SQL na tabela temporária
dim_atores = spark.sql(dim_func_sql)

contagem_de_dados = dim_atores.count()
contagem_de_colunas = len(dim_atores.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

dim_atores.show()

O DataFrame tem 7 colunas.
O DataFrame tem 488 linhas (dados).


+---------+---------+--------------------+-----+---------------+-----------------+--------------------+
| id_filme|  id_nome|              titulo|ordem|      categoria|       personagem|            trabalho|
+---------+---------+--------------------+-----+---------------+-----------------+--------------------+
|tt0063803|nm1058895|Who's That Knocki...|   10|cinematographer|               \N|director of photo...|
|tt0063803|nm0000217|Who's That Knocki...|   10|cinematographer|               \N|director of photo...|
|tt0063803|nm1058895|Who's That Knocki...|    1|          actor|         ["J.R."]|                  \N|
|tt0063803|nm0000217|Who's That Knocki...|    1|          actor|         ["J.R."]|                  \N|
|tt0063803|nm1058895|Who's That Knocki...|    2|        actress|         ["Girl"]|                  \N|
|tt0063803|nm0000217|Who's That Knocki...|    2|        actress|         ["Girl"]|                  \N|
|tt0063803|nm1058895|Who's That Knocki...|    3|        actress|

In [ ]:
# EXPORTANDO TABELA

In [20]:
from datetime import datetime

current_date = datetime.now().strftime("%Y/%m/%d")

# Definindo a variável do Output

output_path = f"file:/home/hadoop/imdb/refined/dim_atores"

# Salvar o DataFrame como um arquivo Parquet

dim_atores.coalesce(1).write.saveAsTable(name="dim_atores",mode="overwrite",path=output_path,format="parquet")

print("Os dados foram salvos com sucesso como arquivo Parquet.")

Os dados foram salvos com sucesso como arquivo Parquet.


In [ ]:
# INFO FILMES_vOTOS

In [30]:
filmes_votos = spark.read.parquet("file:/home/hadoop/imdb/parquet/filmes_votos/2023/11/01/*.parquet")
filmes_votos.printSchema()
filmes_votos.show(10)
contagem_de_dados = filmes_votos.count()
contagem_de_colunas = len(filmes_votos.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

root
 |-- id_filme: string (nullable = true)
 |-- avaliacao_media: string (nullable = true)
 |-- total_votos: string (nullable = true)

+---------+---------------+-----------+
| id_filme|avaliacao_media|total_votos|
+---------+---------------+-----------+
|tt0000001|            5.7|       2003|
|tt0000002|            5.8|        269|
|tt0000003|            6.5|       1898|
|tt0000004|            5.5|        178|
|tt0000005|            6.2|       2682|
|tt0000006|            5.0|        183|
|tt0000007|            5.4|        839|
|tt0000008|            5.4|       2147|
|tt0000009|            5.3|        207|
|tt0000010|            6.9|       7331|
+---------+---------------+-----------+
only showing top 10 rows

O DataFrame tem 3 colunas.
O DataFrame tem 1363146 linhas (dados).


In [ ]:
# join com fatos fillmes

In [31]:
filmes_votos.createOrReplaceTempView("dim_popularidade")

dim_votos_sql = """
SELECT
    f.id_filme,
    f.id_nome,
    f.titulo,
    de.avaliacao_media,
    de.total_votos
    
FROM
    fato_filmes f
JOIN
    dim_popularidade de
ON
    f.id_filme = de.id_filme
"""

dim_popularidade = spark.sql(dim_votos_sql)

contagem_de_dados = dim_popularidade.count()
contagem_de_colunas = len(dim_popularidade.columns)

# contagem de colunas
print(f"O DataFrame tem {contagem_de_colunas} colunas.")

# contagem de linhas
print(f"O DataFrame tem {contagem_de_dados} linhas (dados).")

dim_popularidade.show()

O DataFrame tem 5 colunas.
O DataFrame tem 49 linhas (dados).
+---------+---------+--------------------+---------------+-----------+
| id_filme|  id_nome|              titulo|avaliacao_media|total_votos|
+---------+---------+--------------------+---------------+-----------+
|tt0063803|nm1058895|Who's That Knocki...|            6.6|       9794|
|tt0063803|nm0000217|Who's That Knocki...|            6.6|       9794|
|tt0068309|nm0181019|       Boxcar Bertha|            6.0|      10493|
|tt0068309|nm0181020|       Boxcar Bertha|            6.0|      10493|
|tt0068309|nm0718639|       Boxcar Bertha|            6.0|      10493|
|tt0070379|nm0552731|        Mean Streets|            7.2|     116098|
|tt0070379|nm0000217|        Mean Streets|            7.2|     116098|
|tt0071115|nm0315205|Alice Doesn't Liv...|            7.3|      26062|
|tt0075314|nm0001707|         Taxi Driver|            8.2|     890607|
|tt0076451|nm0552731|  New York, New York|            6.6|      21337|
|tt0076451|nm07

In [ ]:
# exportando

In [32]:
from datetime import datetime
current_date = datetime.now().strftime("%Y/%m/%d")

# Definindo a variável do Output

output_path = f"file:/home/hadoop/imdb/refined/dim_popularidade"

# Salvar o DataFrame como um arquivo Parquet

dim_popularidade.coalesce(1).write.saveAsTable(name="dim_popularidade",mode="overwrite",path=output_path,format="parquet")

print("Os dados foram salvos com sucesso como arquivo Parquet.")

Os dados foram salvos com sucesso como arquivo Parquet.
